In [2]:
import pandas as pd
import numpy as np
import json

In [30]:
def ecoplate_importer(fname,
                      sheet_json,
                      sample_info,
                      metab_json,
                      average_blanks=False,
                      zero_negatives=True,
                      plate_flip=False):
    '''
    Importer to convert EcoPlate data in Excel format to a tidy DataFrame.

    Params
    ______

    fname : str
        Path to an Excel file containing  EcoPlate data. The function expects
        that every Excel file represents a timepoint, and every sheet within the 
        file represents a plate. Plate data should be exported such that somewhere
        in the sheet is a table with the columns "Well" (containing the well numbers)
        and "590" (containing the absorbance values). It is not critical on which line
        the table starts; the function will attempt to find a line that matches this
        format.

    sheet_json : str
        Path to a JSON file containing sample information for each sheet. The keys of 
        the JSON should be the sheet names (e.g., "Plate 1 - Sheet1") and the values
        should be the sample name (e.g., "25D_W_1"). If the sample name contains
        multiple pieces of relevant information (in this case condition, water/larvae,
        and replicate), the pieces should be separated by underscores. There are no limits
        on the amount of information you can store in the sample name, but all sample names
        must contain the same number of underscores.

    sample_info : tuple or None
        A tuple containing the desired column names for infomation derived from sample
        names. For samples with the format "25D_W_1", sample_info might be ("diapause",
        "water","replicate"). If None, the columns will be named "s1", "s2", etc.

    metab_json : str
        Path to a JSON file in which keys are the plate well positions and values are the 
        metabolites. The JSON provided in the "resources" folder of this repo is derived
        from this file: 
        https://www.biolog.com/wp-content/uploads/2023/08/00A-012-Rev-F-EcoPlate-IFU.pdf

    average_blanks : Bool
        If True, all Water well 590 values will be averaged and used as the blank for the 
        entire plate. If False, the A1 Water well will be used to blank columns 1-4, A5 
        for columns 5-8, and A9 for columns 9-12. Default: False
    
    zero_negatives : Bool
        If True, any blanked 590 absorbance values below zero will instead be set to zero/
        If False, negative values will remain negative. Default: True

    plate_flip : Bool
        Set to True if the plate was read such that H12 was accidentally put in the A1
        position. The function will remap all of the values to the correct wells before
        labeling metabolites.

    Output
    ______

    A Pandas DataFrame containing columns for well, metabolite, raw 590 value, blanked
    590 value, and any sample information.     
    
    '''
    # Get sample ID and metabolite info
    with open(sheet_json,'r') as f:
        pid = json.load(f)

    with open(metab_json,'r') as f:
        eco = json.load(f)

    # Identify blank wells
    water = [k for k,v in eco.items() if v == 'Water']

    # Get sheets
    sheets = pd.ExcelFile(fname).sheet_names

    # Extract data for each sheet
    df_ls = []
    for s in sheets:
        # Identify header row to get number of rows to skip
        df = pd.read_excel(fname,
                           sheet_name=s)
        skiprows = None
        for i in df.index:
            if 'Well' in list(df.loc[i,:]):
                skiprows=i+1
            else:
                pass

        # Verify that header was identified, otherwise raise an error
        try:
            int(skiprows)
        except:
            raise Exception('Header row not found for '+sheet_name)

        # Import tidy data
        df = pd.read_excel('exampleData/24h_example.xlsx',
                   sheet_name=sheet_name,
                   skiprows=skiprows)
        df = df.dropna(axis=1)
        df.columns = [str(i) for i in df.columns]

        # Correct plate flipping
        if plate_flip:
            orig_rows = [chr(65+i) for i in range(8)]
            orig_cols = [str(i) for i in np.arange(1,13)]
            
            df['orig_row'] = [i[0] for i in df['Well']]
            df['orig_col'] = [i[1:] for i in df['Well']]
            
            df['new_row'] = [orig_rows[::-1][orig_rows.index(r)] for r in df['orig_row']]
            df['new_col'] = [orig_cols[::-1][orig_cols.index(r)] for r in df['orig_col']]
            
            df['Well'] = df['new_row'] + df['new_col']
            df = df.drop(['orig_row','orig_col','new_row','new_col'], axis=1)
        else:
            pass 

        # Add metabolites
        df['metab'] = [eco[i] for i in df['Well']]

        # Add timepoints
        basename = fname.split('/')[-1]
        timepoint = basename.split('_')[0]
        df['timepoint'] = [timepoint for i in df.index]

        # Add sample information
        subdict = pid[timepoint]
        sample = subdict[s]
        subinfo = sample.split('_')

        df['sample'] = [sample for i in df.index]

        if sample_info:
            sdict = dict(zip(sample_info, subinfo))
        else:
            sdict = dict(zip(['s'+i for i,v in enumerate(subinfo)], 
                             subinfo))

        for col, info in sdict.items():
            df[col] = [info for i in df.index]

        #  Add blanks
        if average_blanks:
            avblank = np.mean(df[df['Well'].isin(water)]['590'])
            df['blanked_590'] = df['590'] - avblank
        else:
            df['column'] = [i[1:] for i in df['Well']]
            techreps = np.append(np.arange(1,12,4),13)
            for i, w in enumerate(water):
                blank = df.set_index('Well').loc[w,'590']
                cols = [str(i) for i in range(techreps[i],techreps[i+1])]
                subdf = df[df['column'].isin(cols)].copy()
                inds = subdf.index
                df.loc[inds,'blanked_590'] = subdf['590'] - blank
            df = df.drop('column', axis=1)
        # Zero negatives
        if zero_negatives:
            df['blanked_590'] = df['blanked_590'].clip(0)
        else:
            pass
                
        df_ls.append(df)

    # Concatenate, reindex, return
    df_all = pd.concat(df_ls)
    df_all.reset_index(drop=True, inplace=True)

    return df_all

In [37]:
df = ecoplate_importer('exampleData/24h_example.xlsx',
                  'exampleData/plateIDs.json',
                  ('diapause','specimen','rep'),
                  'resources/ecoPlate.json',
                  average_blanks=False,
                  zero_negatives=True,
                  plate_flip=False)

df.head()

,Well,590,metab,timepoint,sample,diapause,specimen,rep,blanked_590
0,A1,0.310,Water,24h,4_L_1,4,L,1,0.0
1,A5,0.271,Water,24h,4_L_1,4,L,1,0.0
2,A9,0.334,Water,24h,4_L_1,4,L,1,0.0
3,A2,0.206,β-Methyl-D-Glucoside,24h,4_L_1,4,L,1,0.0
4,A6,0.221,β-Methyl-D-Glucoside,24h,4_L_1,4,L,1,0.0


In [38]:
df[df['metab'] == 'Putrescine']

,Well,590,metab,timepoint,sample,diapause,specimen,rep,blanked_590
93,H8,0.134,Putrescine,24h,4_L_1,4,L,1,0.0
94,H12,0.127,Putrescine,24h,4_L_1,4,L,1,0.0
95,H4,0.140,Putrescine,24h,4_L_1,4,L,1,0.0
189,H8,0.134,Putrescine,24h,4_L_2,4,L,2,0.0
190,H12,0.127,Putrescine,24h,4_L_2,4,L,2,0.0
...,...,...,...,...,...,...,...,...,...
2206,H12,0.127,Putrescine,24h,25E_W_2,25E,W,2,0.0
2207,H4,0.140,Putrescine,24h,25E_W_2,25E,W,2,0.0
2301,H8,0.134,Putrescine,24h,25E_W_1,25E,W,1,0.0
2302,H12,0.127,Putrescine,24h,25E_W_1,25E,W,1,0.0
